In [1]:
import os
import glob
import json
import numpy as np
import cv2
from keras.applications.resnet50 import preprocess_input
from keras.models import load_model
np.random.seed(251)

# Below is needed to load model
from keras_efficientnets import EfficientNetB0

Using TensorFlow backend.


In [2]:
##################################### NEW4_trial4 #####################################
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_openpose = 'openpose_output'
dir_json = 'json_inference_test'

NUM_OF_FRAME = 30

# Parameters used in the manual optical flow
color_red = [255, 0, 0]
color_green = [0, 255, 0]
color_blue = [0, 0, 255]
color_yellow = [255, 255, 0]
color_pink = [255, 0, 255]
color_teal = [0, 255, 255]
color_white = [255, 255, 255]
COLOR_MIN = 50
color_step = (255 - COLOR_MIN) / NUM_OF_FRAME

selected_feature_dict = dict()
# add head detection
selected_feature_dict['body_0'] = (color_white, 2)
selected_feature_dict['body_3'] = (color_red, 2)
selected_feature_dict['body_4'] = (color_red, 2)
selected_feature_dict['body_6'] = (color_green, 2)
selected_feature_dict['body_7'] = (color_green, 2)
# different color for thumbs
selected_feature_dict['lefthand_4'] = (color_blue, 1)
selected_feature_dict['lefthand_8'] = (color_yellow, 1)
selected_feature_dict['lefthand_12'] = (color_yellow, 1)
selected_feature_dict['lefthand_16'] = (color_yellow, 1)
selected_feature_dict['lefthand_20'] = (color_yellow, 1)
# different color for thumbs
selected_feature_dict['righthand_4'] = (color_pink, 1)
selected_feature_dict['righthand_8'] = (color_teal, 1)
selected_feature_dict['righthand_12'] = (color_teal, 1)
selected_feature_dict['righthand_16'] = (color_teal, 1)
selected_feature_dict['righthand_20'] = (color_teal, 1)

# Parameters needed for model scoring
model_file = 'Efficientnet_model_weights_NEW4_trial4.h5'
model_saved = load_model(os.path.join(project_path, model_file))

MODEL_PREDICTION_THRESHOLD = 0.5

class_list = ['AGAIN', 'ALL', 'AWKWARD', 'BASEBALL', 'BEHAVIOR', 'CAN', 'CHAT', 'CHEAP', 
              'CHEAT', 'CHURCH', 'COAT', 'CONFLICT', 'COURT', 'DEPOSIT', 'DEPRESS', 
              'DOCTOR', 'DRESS', 'ENOUGH', 'NEG']
def conv_index_to_vocab(ind):
    temp_dict = dict(enumerate(class_list))
    return temp_dict[ind]

In [7]:
# TESTING ONLY
dir_json_original = 'json_NEW'
original_file_lst = [f for f in os.listdir(os.path.join(project_path, dir_openpose, dir_json_original))] 
counter = -1

In [8]:
# create the json directory if it doesn't exist
if not os.path.exists(os.path.join(project_path, dir_openpose, dir_json)):
    os.makedirs(os.path.join(project_path, dir_openpose, dir_json))

# Remove all the files in dir_json
files = glob.glob(os.path.join(project_path, dir_openpose, dir_json, '*'))
for f in files:
    os.remove(f)

In [9]:
while True:
    ######################### WEBCAM + OPENPOSE CODE BELOW #########################
    temp_f = original_file_lst[counter]
    temp_f_new = str(counter).zfill(12) + '_' + temp_f.split('_')[-1]
    !copy {os.path.join(project_path, dir_openpose, dir_json_original, temp_f)} {os.path.join(project_path, dir_openpose, dir_json, temp_f_new)}
    
    counter += 1
    print(counter)
    if counter > 100:
        break
    ######################### WEBCAM + OPENPOSE CODE ABOVE #########################
        
    full_file_lst = [f for f in os.listdir(os.path.join(project_path, dir_openpose, dir_json)) if f.endswith('.json')]
    if len(full_file_lst) < NUM_OF_FRAME:
        print('skip')
        continue
    json_files_lst = full_file_lst[-NUM_OF_FRAME:]
    
    video_feature_dict = dict()
    for json_f in json_files_lst:
        with open(os.path.join(project_path, dir_openpose, dir_json, json_f)) as ff:
            json_code = json.load(ff)
        
        # This assume there is only one person
        body_raw_lst = json_code['people'][0]['pose_keypoints_2d']
        left_hand_raw_lst = json_code['people'][0]['hand_left_keypoints_2d']
        right_hand_raw_lst = json_code['people'][0]['hand_right_keypoints_2d']
        
        for feat in list(selected_feature_dict.keys()):
            feat_num = int(feat.split('_')[1])
            feat_value =  video_feature_dict.get(feat, [])
            if 'body' in feat:
                feat_value.append(body_raw_lst[3*feat_num:3*(feat_num+1)])
                video_feature_dict[feat] = feat_value
            elif 'lefthand' in feat:
                feat_value.append(left_hand_raw_lst[3*feat_num:3*(feat_num+1)])
                video_feature_dict[feat] = feat_value
            elif 'righthand' in feat:
                feat_value.append(right_hand_raw_lst[3*feat_num:3*(feat_num+1)])
                video_feature_dict[feat] = feat_value
    
    ######################### Update the code below #########################
    mask = np.zeros((480, 640, 3))
    # mask = np.zeros_like(frame)
    ######################### Update the code above #########################
    
    for (k, v) in video_feature_dict.items():
        # (color, thickness)
        (c, t) = selected_feature_dict[k] 
        color_counter = 0
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            conf_1 = points[2]
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                c = [max(color_i-color_step,0) for color_i in c]
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), c, t)
                color_counter += 1
                x_0 = x_1
                y_0 = y_1
    x = cv2.resize(mask, (224,224))
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    y_pred = model_saved.predict(x)
    if np.max(y_pred) < MODEL_PREDICTION_THRESHOLD:
        print('?')
    else:
        print('Prediction: ', conv_index_to_vocab(np.argmax(y_pred)))


        1 file(s) copied.
0
skip
        1 file(s) copied.
1
skip
        1 file(s) copied.
2
skip
        1 file(s) copied.
3
skip
        1 file(s) copied.
4
skip
        1 file(s) copied.
5
skip
        1 file(s) copied.
6
skip
        1 file(s) copied.
7
skip
        1 file(s) copied.
8
skip
        1 file(s) copied.
9
skip
        1 file(s) copied.
10
skip
        1 file(s) copied.
11
skip
        1 file(s) copied.
12
skip
        1 file(s) copied.
13
skip
        1 file(s) copied.
14
skip
        1 file(s) copied.
15
skip
        1 file(s) copied.
16
skip
        1 file(s) copied.
17
skip
        1 file(s) copied.
18
skip
        1 file(s) copied.
19
skip
        1 file(s) copied.
20
skip
        1 file(s) copied.
21
skip
        1 file(s) copied.
22
skip
        1 file(s) copied.
23
skip
        1 file(s) copied.
24
skip
        1 file(s) copied.
25
skip
        1 file(s) copied.
26
skip
        1 file(s) copied.
27
skip
        1 file(s) copied.
28
skip
        1 file(s) copied